# Generating Models
This notebook demonstrates how to generate a model from a compact, string representation of the transcription factors for a gene.

In [1]:
%matplotlib inline

import gene_network as gn

ModuleNotFoundError: No module named 'gene_network'

## Using Model Maker

In [ ]:
# Create the reaction (including kinetics) for gene7
gene7 = gn.GeneReaction(7)
print(gene7)

In [ ]:
result = gn.GeneReaction.do("7+4")
result.reaction

In [ ]:
result.constants

In [ ]:
result = gn.GeneReaction.do("7+4A-1")
result.reaction

## Generate the reactions in the initial network

The text description of the network is:

``
1+0O+4
2+4
3+6
4-2
6+7A-1
8-1
``

In [ ]:
# Print the variable part of the model, both reactions and constants.
descriptions = ["1+0O+4", "2+4", "3+6", "4-2", 5, "6+7A-1", 7, "8-1"]
mrna_reactions = ""
constants_initializations = ""
for stg in descriptions:
    result = gn.GeneReaction.do(stg)
    mrna_reactions += "%s\n" % result.reaction
    # constants_initializations +=  "%s\n" % result.constants
    for constant in result.constants:
        if (not "d_mRNA" in constant) and (not "L" in constant):
            constants_initializations += "%s = 0;\n" % constant
print(mrna_reactions)
print(constants_initializations)
    

Now compare the printed reactions with the initial model.

### Initial model

``
J1:  => mRNA1; L1 + Vm1*((K1_1*INPUT^H1 + K2_1*P4^H1 + K1_1*K3_1*INPUT^H1*P4^H1)/(1 + K1_1*INPUT^H1 + K2_1*P4^H1 + K1_1*K3_1*INPUT^H1*P4^H1)) - d    _mRNA1*mRNA1;
J2:  => mRNA2; L2 + Vm2*(K1_2*P4^H2/(1 + K1_2*P4^H2)) - d_mRNA2*mRNA2;
J3:  => mRNA3; L3 + Vm3*(K1_3*P6^H3/(1 + K1_3*P6^H3)) - d_mRNA3*mRNA3;
J4:  => mRNA4; L4 + Vm4*(1/(1 + K1_4*P2^H4)) - d_mRNA4*mRNA4;
J5:  => mRNA5; L5 - d_mRNA5*mRNA5;
J6:  => mRNA6; L6 + Vm6*(K1_6*P7^H6/(1 + K1_6*P7^H6 + K2_6*P1^H6 + K1_6*K2_6*P7^H6*P1^H6)) - d_mRNA6*mRNA6;
J7:  => mRNA7; L7 - d_mRNA7*mRNA7;
J8:  => mRNA8; L8 + Vm8*(1/(1 + K1_8*P1^H8)) - d_mRNA8*mRNA8;
``
